In [2]:
from __future__ import print_function

import os
import sys
import time
import json
import requests
import lxml.html
import io
from tqdm import tqdm
from lxml.cssselect import CSSSelector

#used to make the Browser Working
from selenium import webdriver
#Send keycodes to Elements
from selenium.webdriver.common.keys import Keys
#scrape the url's and comments
from bs4 import BeautifulSoup

import re
import datetime
import time
import codecs




/home/mi_air/.local/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:


YOUTUBE_COMMENTS_URL = 'https://www.youtube.com/all_comments?v={youtube_id}'
YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'


In [3]:
str(datetime.date.today())

'2019-04-10'

In [4]:

# The List where the links to the videos are stored
links = set()

comments = list()

homePage = 'https:www.youtube.com'
linksSize = 10
driver = webdriver.Firefox()

output = open("/media/mi_air/0F0B7DDE62EEA81E/youtube/"+str(datetime.date.today())+".txt","w")

def loadFullPage(Timeout):
    reachedbottom = None
    while not reachedbottom:
        #scroll one pane down
        driver.execute_script("window.scrollTo(0,Math.max(document.documentElement.scrollHeight,document.body.scrollHeight,document.documentElement.clientHeight));");
        time.sleep(Timeout)
        #check if the bottom is reached
        a = driver.execute_script("return document.documentElement.scrollTop;")
        b = driver.execute_script("return document.documentElement.scrollHeight - document.documentElement.clientHeight;")
        relativeHeight = a / b
        if(relativeHeight==1):
            reachedbottom = True
def getComments(link):
    driver.get(url='https:youtube.com'+link)
    loadFullPage(1)


def main():
    driver.get(url=homePage)
    enoughLinks = None

    while not enoughLinks:
        loadFullPage(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        for link in soup.find_all("a",class_="yt-simple-endpoint style-scope ytd-grid-video-renderer", href=True):
            if link not in links:
                links.add(link['href'])

        if len(links) < linksSize:
            driver.refresh()
        else:
            #for i in range(len(links)-1000):
                #links.pop()
            enoughLinks = True

    #links.sort()
    for link in links:
        output.write(link)
        output.write("\n")
    output.close()
    print(links)


if __name__ == '__main__':
    main()

{'/watch?v=ew-8i7UUpLY', '/watch?v=oOxDLuTXyCo', '/watch?v=32Rj7dv2IRE', '/watch?v=RtSS0SJE8oE', '/watch?v=N0CGM956z18', '/watch?v=fhCLQNYowcE', '/watch?v=AbXahmBSLTk', '/watch?v=ocekebVtZvw', '/watch?v=JM-GwDh73Wc', '/watch?v=BLHrjzTEr0c', '/watch?v=H6Kl8kheGBg', '/watch?v=cnn-z4U_S50', '/watch?v=F1B9Fk_SgI0', '/watch?v=qr4AaWAkf34', '/watch?v=knBhDpMXsQo', '/watch?v=DEfgiRorfbM', '/watch?v=DVH0CzurtIE', '/watch?v=p1JPKLa-Ofc', '/watch?v=nhcDl3S5sXQ', '/watch?v=TcMBFSGVi1c', '/watch?v=b52lho8lD6Q', '/watch?v=kPg3M4C9N9w', '/watch?v=XlmaJ-yU46U', '/watch?v=U_90XNCBatY', '/watch?v=6pdfriFuFt8', '/watch?v=vw2SaHkGfss', '/watch?v=yzZIS1TtXjw', '/watch?v=q97nD5dOS5M', '/watch?v=31OnT5iSLA0', '/watch?v=3y-O-4IL-PU', '/watch?v=RzfO1FbUCo8', '/watch?v=IRKwwk7CXBQ', '/watch?v=ZD9OkKE0TfA', '/watch?v=35adpxPiNlU', '/watch?v=4yXU8K-9SIw', '/watch?v=nDq6TstdEi8', '/watch?v=3XNDaISqFX8', '/watch?v=3p1fHBNILhM', '/watch?v=W1j28DRcFBQ', '/watch?v=qywZ6lUcNo8', '/watch?v=_XFzT9GMmw8', '/watch?v=hE2Ir

In [ ]:
def find_value(html, key, num_chars=2):
    pos_begin = html.find(key) + len(key) + num_chars
    pos_end = html.find('"', pos_begin)
    return html[pos_begin: pos_end]


def extract_comments(html):
    tree = lxml.html.fromstring(html)
    item_sel = CSSSelector('.comment-item')
    text_sel = CSSSelector('.comment-text-content')
    time_sel = CSSSelector('.time')
    author_sel = CSSSelector('.user-name')

    for item in item_sel(tree):
        yield {'cid': item.get('data-cid'),
               'text': text_sel(item)[0].text_content(),
               'time': time_sel(item)[0].text_content().strip(),
               'author': author_sel(item)[0].text_content()}


def extract_reply_cids(html):
    tree = lxml.html.fromstring(html)
    sel = CSSSelector('.comment-replies-header > .load-comments')
    return [i.get('data-cid') for i in sel(tree)]


def ajax_request(session, url, params, data, retries=10, sleep=20):
    for _ in range(retries):
        response = session.post(url, params=params, data=data)
        if response.status_code == 200:
            response_dict = json.loads(response.text)
            return response_dict.get('page_token', None), response_dict['html_content']
        else:
            time.sleep(sleep)


def download_comments(youtube_id, sleep=1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT

    # Get Youtube page with initial comments
    response = session.get(YOUTUBE_COMMENTS_URL.format(youtube_id=youtube_id))
    html = response.text
    reply_cids = extract_reply_cids(html)

    ret_cids = []
    for comment in extract_comments(html):
        ret_cids.append(comment['cid'])
        yield comment

    page_token = find_value(html, 'data-token')
    session_token = find_value(html, 'XSRF_TOKEN', 4)

    first_iteration = True

    # Get remaining comments (the same as pressing the 'Show more' button)
    while page_token:
        data = {'video_id': youtube_id,
                'session_token': session_token}

        params = {'action_load_comments': 1,
                  'order_by_time': True,
                  'filter': youtube_id}

        if first_iteration:
            params['order_menu'] = True
        else:
            data['page_token'] = page_token

        response = ajax_request(session, YOUTUBE_COMMENTS_AJAX_URL, params, data)
        if not response:
            break

        page_token, html = response

        reply_cids += extract_reply_cids(html)
        for comment in extract_comments(html):
            if comment['cid'] not in ret_cids:
                ret_cids.append(comment['cid'])
                yield comment

        first_iteration = False
        time.sleep(sleep)

    # Get replies (the same as pressing the 'View all X replies' link)
    for cid in reply_cids:
        data = {'comment_id': cid,
                'video_id': youtube_id,
                'can_reply': 1,
                'session_token': session_token}

        params = {'action_load_replies': 1,
                  'order_by_time': True,
                  'filter': youtube_id,
                  'tab': 'inbox'}

        response = ajax_request(session, YOUTUBE_COMMENTS_AJAX_URL, params, data)
        if not response:
            break

        _, html = response

        for comment in extract_comments(html):
            if comment['cid'] not in ret_cids:
                ret_cids.append(comment['cid'])
                yield comment
        time.sleep(sleep)


def main(youtube_id, output, limit=100):

    try:

        if not youtube_id or not output:
            parser.print_usage()
            raise ValueError('you need to specify a Youtube ID and an output filename')

        print('Downloading Youtube comments for video:', youtube_id)
        count = 0
        with io.open(output, 'w', encoding='utf8') as fp:
            for comment in download_comments(youtube_id):
                sys.stdout.write(json.dumps(comment, ensure_ascii=False))
                count += 1
                sys.stdout.write('Downloaded %d comment(s)\r' % count)
                sys.stdout.flush()
                if limit and count >= limit:
                    break
        print('\nDone!')


    except Exception as e:
        print('Error:', str(e))
        sys.exit(1)



In [ ]:
wdir = r'/media/mi_air/0F0B7DDE62EEA81E/youtube/comments'

uids = open(r'/media/mi_air/0F0B7DDE62EEA81E/youtube/'+str(datetime.date.today())+'.txt', 'r', encoding='utf8').readlines()
uids = [i.strip('/watch?v=') for i in uids]
for i in tqdm(uids[:10]):
    uid = i.strip('\n')
    output = os.path.join(wdir, uid+'_'+str(datetime.date.today())+'.txt')
    main(uid, output)


  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:07<01:11,  7.99s/it]


Done!


 20%|██        | 2/10 [00:16<01:05,  8.13s/it]


Done!


 30%|███       | 3/10 [00:24<00:57,  8.15s/it]


Done!
